## Data Analyizer for cs584 course project
Auther: kun wu

## 1. Load and Preprocess

### Load Glove pre-train embedding

In [3]:
import numpy as np
from tqdm.notebook import tqdm

def load_glove_weights():
    glove_weights = {}
    with open('../data/glove.6B.300d.txt', 'r') as fp:
        for line in tqdm(fp):
            line = line.split(' ')
            word = line[0].strip().lower()
            vector = np.array([float(f.strip()) for f in line[1:]])
            glove_weights[word.lower()] = vector
    return glove_weights

In [9]:
def generate_weights(glove_weights, word2idx, embedding_size=300):
    weights = np.zeros((len(word2idx) + 1, embedding_size))
    for word, idx in tqdm(word2idx.items()):
        if word in glove_weights:
            weights[word2idx[word]] = np.array(glove_weights[word])
    
    return weights

### Load data

In [2]:
import pandas as pd

train_df = pd.read_csv('../data/train.csv')

print(f'Data size: {train_df.shape}')

train_df.head()

Data size: (1804874, 45)


,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


### Preprocess

In [5]:
train_df['target'] = train_df['target'].apply(lambda x: 1 if x > 0.5 else 0)
x_train = train_df['comment_text'].astype(str).values
y_train = train_df['target'].values

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)

vocab_size = len(tokenizer.word_counts)
max_len = 0
for seq in x_train:
    max_len = len(seq) if len(seq) > max_len else max_len
    
x_train = pad_sequences(x_train, maxlen=max_len)

print(f'Vocab size: {vocab_size}')
print(f'Max length: {max_len}')

Using TensorFlow backend.


Vocab size: 397708
Max length: 317


In [10]:
glove_weights = load_glove_weights()

In [11]:
weights = generate_weights(glove_weights, tokenizer.word_index)
print(f'weights shape: {weights.shape}')


weights shape: (397709, 300)


In [16]:
from keras.layers import Input, LSTM, Bidirectional, Embedding, Dense
from keras.models import Model

vocab_size, embedding_size = weights.shape
hidden_size = 100

words = Input(shape=(None, ), name='input_layer')
x = Embedding(vocab_size, embedding_size, weights=[weights], trainable=False, name='embedding_layer')(words)
x = Bidirectional(LSTM(hidden_size, return_sequences=False, name='lstm1'))(x)
# x = Bidirectional(LSTM(hidden_size, return_sequences=False), name='lstm2')(x)
x = Dense(100, activation='relu')(x)
out = Dense(1, activation='sigmoid')(x)

model = Model(inputs=words, output=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, None)              0         
_________________________________________________________________
embedding_layer (Embedding)  (None, None, 300)         119312700 
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 119,653,701
Trainable params: 341,001
Non-trainable params: 119,312,700
_________________________________________________________________


/Users/tonywu/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [17]:
batch_size = 128
epochs = 10

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 1443899 samples, validate on 360975 samples
Epoch 1/10
  61056/1443899 [>.............................] - ETA: 3:28:32 - loss: 0.1651 - acc: 0.9466

KeyboardInterrupt: 